In [5]:
import numpy as np
import random
import math
import json
import os
from datetime import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


In [12]:

def compute_vs(p1, p2, v_f, v_AUV):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    Lx, Ly, Lz = x2 - x1, y2 - y1, z2 - z1
    L_mag = math.sqrt(Lx**2 + Ly**2 + Lz**2)
    if L_mag == 0:
        return v_AUV
    cos_beta = Lz / L_mag
    cos_beta = np.clip(cos_beta, -1, 1)
    beta = math.acos(cos_beta)
    inner = np.clip((v_f * cos_beta) / v_AUV, -1, 1)
    angle = beta + math.acos(inner)
    if abs(cos_beta) < 1e-9:
        return v_AUV
    return abs(math.cos(angle) * v_AUV / cos_beta)

def travel_time(path, coords, v_f, v_AUV):
    total_time = 0.0
    if len(path) <= 1:
        return 0.0
    for i in range(len(path) - 1):
        p1, p2 = coords[path[i]], coords[path[i + 1]]
        d = np.linalg.norm(np.array(p2) - np.array(p1))
        v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
        total_time += d / max(v_s, 1e-9)
    # return to start
    p1, p2 = coords[path[-1]], coords[path[0]]
    d = np.linalg.norm(np.array(p2) - np.array(p1))
    v_s = compute_vs(tuple(p1), tuple(p2), v_f, v_AUV)
    total_time += d / max(v_s, 1e-9)
    return total_time


In [13]:
def calculate_number_clusters(nodes, base_station=(200, 200, 400), space_size=400):
    """
    Tính số cụm optimal theo công thức
    K = √(NL / pi*d_tobs)
    
    nodes: tọa độ 3D của các node
    base_station: tọa độ base station (mặc định tại gốc tọa độ)
    """
    N = len(nodes)
    base_pos = np.array(base_station)
    
    # Tính khoảng cách trung bình từ các node đến base station
    distances = np.linalg.norm(nodes - base_pos, axis=1)
    d_tobs = np.mean(distances)
    
    # Áp dụng công thức K = √(N*L / pi*d_tobs)
    K_optimal = np.sqrt(N * space_size / (np.pi * d_tobs))
    K_optimal = max(1, int(np.round(K_optimal)))  # Đảm bảo K >= 1
    
    print(f"N = {N}, d_tobs = {d_tobs:.2f}, K_optimal = {K_optimal}")
    return K_optimal

In [14]:
def cluster_split(nodes, node_ids, node_data=None, r_sen=50, R=20, N=30, max_depth=10, depth=0):
    """
    K-Means
    
    nodes: tọa độ 3D của các node
    node_ids: list id tương ứng của các node
    node_data: dictionary chứa thông tin đầy đủ của nodes (bao gồm energy)
    r_sen: bán kính truyền tải tối đa (Rsep - subgroup threshold), default: 100m
    R: size threshold - số lượng node tối đa trong 1 cụm, default: 20
    N: số lần thực hiện clustering để chọn kết quả tốt nhất, default: 30
    max_depth: độ sâu đệ quy tối đa
    depth: độ sâu hiện tại
    """
    
    # Điều kiện dừng:
    # 1. Size threshold: len(nodes) <= R
    # 2. Subgroup threshold: max distance <= r_sen
    size_ok = len(nodes) <= R
    distance_ok = check_subgroup_threshold(nodes, r_sen)
    
    if (size_ok and distance_ok) or depth >= max_depth:
        center = np.mean(nodes, axis=0)
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]
    
    # Bước 2: Thực hiện K-means clustering N lần và chọn kết quả tốt nhất
    labels, centers, best_T = kmeans_with_best_T(nodes, N)
    
    # Bước 3: Phân chia thành các subgroups và đệ quy
    clusters = []
    for i in range(2):
        # Lọc nodes và node_ids theo label
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        
        # Tạo sub_node_data cho cluster con
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        
        # Đệ quy cho subgroup
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, N, max_depth, depth + 1)
    
    return clusters


In [15]:

def choose_cluster_head(cluster, node_data_dict):
    """
    Chọn cluster head là node gần tâm cụm nhất và có năng lượng cao nhất.
    
    Chiến lược:
    1. Tìm node gần tâm cụm (center) nhất
    2. Nếu có nhiều node cùng khoảng cách, chọn node có năng lượng cao nhất
    
    Parameters:
    - cluster: Dictionary chứa thông tin cụm (nodes, center, node_ids)
    - node_data_dict: Dictionary chứa thông tin năng lượng và vị trí của các node
    
    Returns:
    - node_id của cluster head (node gần center nhất, năng lượng cao nhất)
    """
    node_ids = cluster["node_ids"]
    center = np.array(cluster["center"])
    
    if len(node_ids) == 0:
        return None
    
    # Tìm node gần tâm cụm nhất với năng lượng cao nhất
    best_node = None
    min_distance = float('inf')
    max_energy_at_min_dist = -1
    
    for node_id in node_ids:
        if node_id not in node_data_dict:
            continue
            
        # Lấy vị trí node từ cluster['nodes']
        node_idx = node_ids.index(node_id)
        node_pos = cluster["nodes"][node_idx]
        
        # Tính khoảng cách đến tâm cụm
        distance = np.linalg.norm(np.array(node_pos) - center)
        energy = node_data_dict[node_id].get('residual_energy', 0.0)
        
        # Chọn node gần center nhất, nếu bằng nhau thì chọn node có năng lượng cao hơn
        if distance < min_distance or (distance == min_distance and energy > max_energy_at_min_dist):
            min_distance = distance
            max_energy_at_min_dist = energy
            best_node = node_id
    
    return best_node if best_node is not None else node_ids[0]


In [ ]:
def calculate_objective_function(nodes, labels, centers):
    #  Tính hàm mục tiêu k means
    numerator = 0
    for i in range(2):
        cluster_nodes = nodes[labels == i]
        if len(cluster_nodes) > 0:
            distances = np.linalg.norm(cluster_nodes - centers[i], axis=1)
            numerator += np.sum(distances)
    
    denominator = np.linalg.norm(centers[0] - centers[1])
    
    if denominator == 0:
        return float('inf')
    
    T = numerator / denominator
    return T

def check_subgroup_threshold(nodes, r_sen):
    # Khoảng cách Euclidean tối đa giữa hai node bất kỳ trong cụm <= r_sen
    if len(nodes) <= 1:
        return True
    
    # Tính khoảng cách giữa tất cả các cặp node
    max_distance = 0
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            dist = np.linalg.norm(nodes[i] - nodes[j])
            max_distance = max(max_distance, dist)
    
    return max_distance <= r_sen

def kmeans_with_best_T(nodes, N=30):
    """
    Thực hiện K-means N lần với các tâm khởi tạo ngẫu nhiên khác nhau
    và chọn kết quả có giá trị T nhỏ nhất
    """
    best_T = float('inf')
    best_labels = None
    best_centers = None
    
    for _ in range(N):
        kmeans = KMeans(n_clusters=2, n_init=1)
        labels = kmeans.fit_predict(nodes)
        centers = kmeans.cluster_centers_
        
        T = calculate_objective_function(nodes, labels, centers)
        
        if T < best_T:
            best_T = T
            best_labels = labels.copy()
            best_centers = centers.copy()
    
    return best_labels, best_centers, best_T

def compute_energy(best_time, n_members):
    """
    Tính năng lượng tiêu thụ cho Member Node và Cluster Head.
    
    Parameters:
    - best_time: Thời gian hoàn thành chu kỳ AUV
    - n_members: Số lượng node thành viên thực tế trong cluster (không tính cluster head)
    """
    G, L = 100, 1024
    P_t, P_r, P_idle, DR, DR_i = 1.6e-3, 0.8e-3, 0.1e-3, 4000, 1e6

    # Năng lượng cho Member Node
    E_tx_MN = G * P_t * L / DR
    E_idle_MN = (best_time - G * L / DR) * P_idle
    E_total_MN = E_tx_MN + E_idle_MN

    # Năng lượng cho Cluster Head (nhận từ n_members node, truyền cho AUV)
    E_rx_TN = G * P_r * L * n_members / DR
    E_tx_TN = G * P_t * L * n_members / DR_i
    E_idle_TN = (best_time - (G*L*n_members/DR) - (G*L*n_members/DR_i)) * P_idle
    E_total_TN = E_rx_TN + E_tx_TN + E_idle_TN

    return {
        "Member": {"E_total": E_total_MN},
        "Target": {"E_total": E_total_TN}
    }

def update_energy(all_nodes, clusters, best_time):
    """
    Cập nhật năng lượng cho tất cả các node dựa trên số member thực tế của từng cluster.
    
    Parameters:
    - all_nodes: Dictionary chứa thông tin tất cả các node
    - clusters: Dictionary chứa thông tin các cluster
    - best_time: Thời gian hoàn thành chu kỳ AUV
    """
    for cid, cinfo in clusters.items():
        ch = cinfo.get('cluster_head')
        nodes = cinfo.get('nodes', [])
        
        # Tính số member nodes (không tính cluster head)
        n_members = len([n for n in nodes if n != ch])
        
        # Tính năng lượng cho cluster này với số member thực tế
        energy_report = compute_energy(best_time, n_members)
        
        for nid in nodes:
            if nid not in all_nodes: continue
            if nid == ch:
                all_nodes[nid]['residual_energy'] -= energy_report['Target']['E_total']
            else:
                all_nodes[nid]['residual_energy'] -= energy_report['Member']['E_total']
            all_nodes[nid]['residual_energy'] = max(all_nodes[nid]['residual_energy'], 0.0)

def remove_dead_nodes(all_nodes, clusters):
    """
    Loại bỏ các node đã hết năng lượng và cập nhật lại clusters.
    
    Returns:
    - new_clusters: Dictionary các cluster còn node sống
    - dead: List các node_id đã chết
    """
    dead = [nid for nid, info in list(all_nodes.items()) if info['residual_energy'] <= 0]
    for nid in dead:
        del all_nodes[nid]

    new_clusters = {}
    for cid, cinfo in clusters.items():
        alive_nodes = [nid for nid in cinfo.get('nodes', []) if nid in all_nodes]
        if alive_nodes:
            new_c = dict(cinfo)
            new_c['nodes'] = alive_nodes
            new_clusters[cid] = new_c

    return new_clusters, dead

def nearest_neighbor_path(centers):
    """
    Tìm đường đi ngắn nhất bằng thuật toán Nearest Neighbor (Greedy TSP).
    
    Parameters:
    - centers: List các tọa độ cần thăm (bắt đầu từ base station ở index 0)
    
    Returns:
    - path: List các index theo thứ tự thăm
    """
    n = len(centers)
    if n == 1:
        return [0]
    unvisited = set(range(1, n))
    path = [0]
    current = 0
    coords = np.array(centers)

    while unvisited:
        candidates = sorted(unvisited)
        cand_coords = coords[candidates]
        dists = np.linalg.norm(cand_coords - coords[current], axis=1)
        next_idx = candidates[int(np.argmin(dists))]
        path.append(next_idx)
        unvisited.remove(next_idx)
        current = next_idx

    return path

def recluster(all_nodes, node_positions, r_sen=50, R=20):
    """
    Phân cụm lại toàn bộ các node còn sống.
    
    Parameters:
    - all_nodes: Dictionary các node còn sống
    - node_positions: Dictionary vị trí của các node
    - r_sen: Ngưỡng khoảng cách tối đa trong cụm
    - R: Số lượng node tối đa trong 1 cụm
    
    Returns:
    - clusters: Dictionary các cụm mới
    """
    ids = sorted(list(all_nodes.keys()))
    if len(ids) == 0:
        return {}

    coords = np.array([node_positions[nid] for nid in ids])
    raw_clusters = cluster_split(coords, ids, all_nodes, r_sen=r_sen, R=R)

    clusters = {}
    for i, c in enumerate(raw_clusters):
        center = c['center'].tolist()
        node_ids = c['node_ids']
        ch = choose_cluster_head(c, all_nodes)
        clusters[i] = {'nodes': node_ids, 'center': center, 'cluster_head': ch}

    return clusters

def main():
    """
    Hàm chính mô phỏng mạng cảm biến dưới nước với AUV thu thập dữ liệu.
    """
    input_dir = "input_data"
    output_dir = "result_ga_ch_most_energy"
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(input_dir):
        print(f"❌ Lỗi: Thư mục {input_dir} không tồn tại!")
        return

    files = [f for f in os.listdir(input_dir) if f.endswith('.json')]
    
    if len(files) == 0:
        print(f"❌ Không tìm thấy file dữ liệu nào trong {input_dir}")
        return

    INITIAL_ENERGY = 100.0
    v_f = 1.2
    v_AUV = 3.0
    results_summary = []

    for filename in files:
        input_path = os.path.join(input_dir, filename)
        print(f"\n{'='*60}")
        print(f"=== Đang xử lý file: {filename} ===")
        print(f"{'='*60}")
        
        try:
            with open(input_path, 'r') as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Lỗi đọc file {filename}: {e}")
            continue

        node_positions = {}
        all_nodes = {}
        
        # Xử lý file JSON - danh sách nodes: [{"id": 0, "x": ..., "y": ..., "z": ...}, ...]
        if isinstance(data, list):
            print("📄 Định dạng: Danh sách nodes")
            for node in data:
                nid = node['id']
                all_nodes[nid] = {
                    'initial_energy': node.get('initial_energy', INITIAL_ENERGY),
                    'residual_energy': node.get('residual_energy', INITIAL_ENERGY)
                }
                node_positions[nid] = (node['x'], node['y'], node['z'])
        else:
            print(f"❌ Cấu trúc file {filename} không được hỗ trợ (cần list of nodes)")
            continue

        total_nodes = len(all_nodes)
        print(f"📊 Tổng số node ban đầu: {total_nodes}")

        # Phân cụm lần đầu tiên
        print("\n🔄 Đang phân cụm lần đầu tiên...")
        initial_clusters = recluster(all_nodes, node_positions)
        
        # Lưu kết quả phân cụm lần đầu
        clusters_output = {}
        for cid, cinfo in initial_clusters.items():
            clusters_output[cid] = {
                'cluster_id': cid,
                'nodes': cinfo['nodes'],
                'center': cinfo['center'],
                'cluster_head': cinfo['cluster_head'],
                'num_nodes': len(cinfo['nodes'])
            }
        
        initial_cluster_file = os.path.join(output_dir, f"initial_clusters_{filename}")
        with open(initial_cluster_file, "w", encoding='utf-8') as f:
            json.dump(clusters_output, f, indent=4, ensure_ascii=False)
        
        print(f"✅ Đã lưu phân cụm ban đầu: {initial_cluster_file}")
        print(f"   - Số cụm: {len(initial_clusters)}")
        print(f"   - Số node/cụm: {[len(c['nodes']) for c in initial_clusters.values()]}")

        cycle = 0
        alive_log = []
        energy_log = []

        # Vòng lặp mô phỏng
        while True:
            cycle += 1
            alive_log.append(len(all_nodes))
            total_energy = sum(all_nodes[n]['residual_energy'] for n in all_nodes)
            energy_log.append(total_energy)

            alive_ratio = len(all_nodes)/total_nodes if total_nodes > 0 else 0
            print(f"\n--- Cycle {cycle} --- | Alive: {alive_ratio*100:.2f}% ({len(all_nodes)}/{total_nodes}) | Energy: {total_energy:.2f}J")

            if alive_ratio < 0.9:
                print("🛑 Dừng mô phỏng vì < 90% node còn sống")
                break

            # Phân cụm lại
            clusters = recluster(all_nodes, node_positions)
            if len(clusters) == 0:
                print("🛑 Không còn node -> Dừng")
                break

            # Tạo đường đi cho AUV
            sorted_keys = sorted(clusters.keys())
            centers = [(0, 0, 0)]  # Base station
            for k in sorted_keys:
                centers.append(tuple(clusters[k]['center']))

            path_indices = nearest_neighbor_path(centers)
            best_time = travel_time(path_indices, centers, v_f, v_AUV)
            
            # Cập nhật năng lượng
            update_energy(all_nodes, clusters, best_time)
            clusters, dead_nodes = remove_dead_nodes(all_nodes, clusters)
            
            if dead_nodes:
                print(f"   ⚡ {len(dead_nodes)} node(s) đã hết năng lượng")

        # Lưu kết quả JSON
        meta = {
            'input_file': filename,
            'initial_total_nodes': total_nodes,
            'cycles_completed': cycle - 1,
            'final_alive_nodes': len(all_nodes),
            'final_alive_ratio': len(all_nodes)/total_nodes if total_nodes > 0 else 0
        }
        
        output_json = os.path.join(output_dir, f"result_{filename}")
        with open(output_json, "w") as f:
            json.dump(meta, f, indent=4)

        results_summary.append((filename, cycle - 1))
        print(f"\n✅ File {filename}: {cycle - 1} cycles hoàn thành")

        # Plot: Alive nodes per cycle
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(alive_log)), alive_log, marker='o', linewidth=2)
        plt.title(f"Số node sống theo chu kỳ - {filename}", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Nodes alive", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"alive_nodes_{filename}.png"), dpi=150)
        plt.close()

        # Plot: Total energy per cycle
        plt.figure(figsize=(10, 6))
        plt.plot(range(len(energy_log)), energy_log, marker='s', linewidth=2, color='orange')
        plt.title(f"Năng lượng toàn mạng theo chu kỳ - {filename}", fontsize=14, fontweight='bold')
        plt.xlabel("Chu kỳ", fontsize=12)
        plt.ylabel("Total energy (J)", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"energy_{filename}.png"), dpi=150)
        plt.close()

    # Summary plot
    if results_summary:
        labels = [x[0] for x in results_summary]
        values = [x[1] for x in results_summary]
        plt.figure(figsize=(10, 6))
        plt.plot(labels, values, marker='o', linewidth=2, markersize=8)
        plt.title("AUV cycles completed per dataset", fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel("Cycles", fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "summary_cycles.png"), dpi=150)
        plt.close()
        
        print(f"\n{'='*60}")
        print(f"✅ Hoàn thành! Kết quả đã lưu tại: {output_dir}")
        print(f"{'='*60}")
    else:
        print("\n⚠️  Không có kết quả nào được tạo ra")


In [22]:
if __name__ == '__main__':
    main()



=== Đang xử lý file: nodes_200.json ===
📄 Định dạng: Danh sách nodes
📊 Tổng số node ban đầu: 200

🔄 Đang phân cụm lần đầu tiên...
✅ Đã lưu phân cụm ban đầu: result_ga_ch_most_energy\initial_clusters_nodes_200.json
   - Số cụm: 33
   - Số node/cụm: [3, 5, 11, 9, 8, 6, 4, 11, 3, 5, 2, 12, 6, 8, 11, 2, 11, 4, 3, 6, 6, 11, 2, 1, 3, 3, 12, 4, 10, 3, 10, 2, 3]

--- Cycle 1 --- | Alive: 100.00% (200/200) | Energy: 20000.00J
✅ Đã lưu phân cụm ban đầu: result_ga_ch_most_energy\initial_clusters_nodes_200.json
   - Số cụm: 33
   - Số node/cụm: [3, 5, 11, 9, 8, 6, 4, 11, 3, 5, 2, 12, 6, 8, 11, 2, 11, 4, 3, 6, 6, 11, 2, 1, 3, 3, 12, 4, 10, 3, 10, 2, 3]

--- Cycle 1 --- | Alive: 100.00% (200/200) | Energy: 20000.00J

--- Cycle 2 --- | Alive: 100.00% (200/200) | Energy: 19971.37J

--- Cycle 2 --- | Alive: 100.00% (200/200) | Energy: 19971.37J

--- Cycle 3 --- | Alive: 100.00% (200/200) | Energy: 19949.17J

--- Cycle 3 --- | Alive: 100.00% (200/200) | Energy: 19949.17J

--- Cycle 4 --- | Alive: 100.0